In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm as tqdm
import urllib.request
import os
from concurrent.futures import ThreadPoolExecutor

#у каждого студента свой вариант
VARIANT = 100
VARIANT_URL = f'https://storage.yandexcloud.net/plates/tasks_spring_2021/{VARIANT}.csv'

#создаем папку data и скачиваем файл с ссылками на изображения
os.makedirs('data', exist_ok = True)
urllib.request.urlretrieve(VARIANT_URL, f'data/{VARIANT}.csv');

df = pd.read_csv(f'data/{VARIANT}.csv')

def download_img(url):
    name = url.split('/')[-1]
    path_img = f'data/images/{name}'
    if not os.path.exists(path_img):
        urllib.request.urlretrieve(url, path_img);

#создаем папку images и скачиваем туда изображения
os.makedirs('data/images', exist_ok = True)
list_url = list(df['url'])
with ThreadPoolExecutor(10) as executor:
        results = list(tqdm(executor.map(download_img, list_url), total=len(list_url)))

In [2]:
#ваша задача - прогнать все 2000 изображений через нейронную сеть и получить predict и уверенность в ответе
df.head(3)

,id,url
0,30100.jpg,https://storage.yandexcloud.net/plates/30100.jpg
1,54865.jpg,https://storage.yandexcloud.net/plates/54865.jpg
2,40585.jpg,https://storage.yandexcloud.net/plates/40585.jpg


In [3]:
#если у вас не получается самостоятельно прогнать, напишите мне, я вам выдам файл с предсказаниями
#но в этом случае вы не получите 2 балла за первый пункт задания
df = pd.read_csv(f'data/{VARIANT}_with_prediction.csv')

#будем считать, что нейронка уверена в ответе, если "accuracy" > 0.9995
df['is_good_prediction'] = (df['confidence'] > 0.9995).astype(int)
df.head()

,id,url,predict,confidence,is_good_prediction
0,30100.jpg,https://storage.yandexcloud.net/plates/30100.jpg,У004МУ30,0.999935,1
1,54865.jpg,https://storage.yandexcloud.net/plates/54865.jpg,М998МР77,0.999910,1
2,40585.jpg,https://storage.yandexcloud.net/plates/40585.jpg,NaN,NaN,0
3,34801.jpg,https://storage.yandexcloud.net/plates/34801.jpg,Р142ХО26,0.999935,1
4,3061.jpg,https://storage.yandexcloud.net/plates/3061.jpg,А777КН67,0.999933,1


In [4]:
df['is_good_prediction'].value_counts()

1    1561
0     439
Name: is_good_prediction, dtype: int64

В 1561 номерах наша нейрнонная сеть уверена.  
Для этих номеров в качестве ответа выдаем predict от нейронной сети.

В 439 номерах наша нейронная сеть не уверена. Отдадим эти изображения на разметку в краудсорсинг. 

In [5]:
from toloka_api import TolokaProcessingApi
import json

#запишем в переменную toloka_token наш токен для работы с API Яндекс.Толоки
toloka_token = json.loads(open("/var/www/nomeroff-net/romankucev/.ssh/tokens_toloka.txt").read())['IE']

toloka_api = TolokaProcessingApi(toloka_token)

In [6]:
#создадим проект из шаблона project_plates_19_10_20.json
project_plates = json.loads(open("project_templates/project_plates_19_10_20.json").read())
project_id = toloka_api.create_toloka_project(project_plates)

New project was created. New project id:  50431
https://toloka.yandex.ru/requester/project/50431


In [7]:
#выбираем изображения для разметки в Яндекс.Толоке
df_toloka = df[df['is_good_prediction'] == 0]

In [8]:
#переименовываем стоблцы
df_toloka = df_toloka.rename(columns={'url':'INPUT:image','predict':'INPUT:predict'})

In [9]:
#сохраняем файл с заданиями, который уже через UI загрузим в наш пул
os.makedirs('data/tsv', exist_ok = True)
df_toloka[['INPUT:image','INPUT:predict']].to_csv('data/tsv/df_toloka.tsv', sep='\t', index=None)

In [10]:
#для того, чтобы вам было проше, я разметил обучащие задания
urllib.request.urlretrieve('https://storage.yandexcloud.net/plates/obuch/obuch_tasks.tsv',
                           'data/tsv/obuch_tasks.tsv');

In [11]:
#для того, чтобы вам было проще, я составил вам немного контрольных заданий
urllib.request.urlretrieve('https://storage.yandexcloud.net/plates/control_tasks/control_tasks.tsv',
                           'data/tsv/control_tasks.tsv');

In [12]:
#запускаем пул, загружаем задания, размечаем.
#скачиваем разметку в файл res.tsv
res = pd.read_csv(f'data/tsv/res.tsv',sep = '\t')
res = res[~res['INPUT:image'].isnull()]

In [13]:
#посмотрим ответы исполнителей
for i in res['INPUT:image'].unique():
    print(i)
    tmp = res[res['INPUT:image'] == i]
    print(tmp['OUTPUT:output'].value_counts().to_string(),'\n');

https://storage.yandexcloud.net/plates/46378.jpg
МН007С23    3 

https://storage.yandexcloud.net/plates/14362.jpg
НЕ309953    3 

https://storage.yandexcloud.net/plates/41633.jpg
О080977    3 

https://storage.yandexcloud.net/plates/20008.jpg
Х575РС56    3 

https://storage.yandexcloud.net/plates/46622.jpg
ВА139416    3 

https://storage.yandexcloud.net/plates/15704.jpg
А158946    3 

https://storage.yandexcloud.net/plates/control_tasks/2037_т444та66_т444та6_0.8459566235542297.jpg
Т444ТА66    3 

https://storage.yandexcloud.net/plates/10190.jpg
А151899    3 

https://storage.yandexcloud.net/plates/48282.jpg
У727ТВ47    3 

https://storage.yandexcloud.net/plates/3557.jpg
В345СТ05    3 

https://storage.yandexcloud.net/plates/16056.jpg
Е125МВ45    3 

https://storage.yandexcloud.net/plates/33125.jpg
1507ВА15    3 

https://storage.yandexcloud.net/plates/35900.jpg
К947КК44    2
Н947КК44    1 

https://storage.yandexcloud.net/plates/control_tasks/992_а031нс01_х871тт17_0.5963855981826782.jp

М520АН124    3 

https://storage.yandexcloud.net/plates/28001.jpg
ВТ351650    3 

https://storage.yandexcloud.net/plates/11362.jpg
Р257УМ39    3 

https://storage.yandexcloud.net/plates/42929.jpg
О136ОО71    3 

https://storage.yandexcloud.net/plates/23643.jpg
3519АР50    2
К271В       1 

https://storage.yandexcloud.net/plates/29490.jpg
Х245КЕ142    3 

https://storage.yandexcloud.net/plates/39750.jpg
Х556КХ33    3 

https://storage.yandexcloud.net/plates/control_tasks/1599_вв865878_в8865в78_0.5032516717910767.jpg
ВВ865878    1 

https://storage.yandexcloud.net/plates/15146.jpg
О001СО99    3 

https://storage.yandexcloud.net/plates/control_tasks/2079_с222рк750_с222ру750_0.8201933503150941.jpg
С222РК750    3 

https://storage.yandexcloud.net/plates/53921.jpg
В037MС126    1
В037мС126    1
В037МС126    1 

https://storage.yandexcloud.net/plates/6409.jpg
А757АА159    3 

https://storage.yandexcloud.net/plates/29126.jpg
АР136086    3 

https://storage.yandexcloud.net/plates/35136.jpg
Х944Н

АМ363533    3 

https://storage.yandexcloud.net/plates/16867.jpg
О451УС55    3 

https://storage.yandexcloud.net/plates/32849.jpg
КВ97314    3 

https://storage.yandexcloud.net/plates/49538.jpg
Т826НМ31    3 

https://storage.yandexcloud.net/plates/32952.jpg
М657АМ67    3 

https://storage.yandexcloud.net/plates/21266.jpg
С555ЕА161    3 

https://storage.yandexcloud.net/plates/control_tasks/1767_т452кв51_т452ко51_0.9733715653419496.jpg
Т452КВ51    1 

https://storage.yandexcloud.net/plates/42456.jpg
Е607СК01    3 

https://storage.yandexcloud.net/plates/59718.jpg
6815АС50    2
АС681550    1 

https://storage.yandexcloud.net/plates/36703.jpg
АН016232    3 

https://storage.yandexcloud.net/plates/54730.jpg
Е233ЕУ138    3 

https://storage.yandexcloud.net/plates/21182.jpg
О095МЕ22    3 

https://storage.yandexcloud.net/plates/1147.jpg
С738МТ70    3 

https://storage.yandexcloud.net/plates/22138.jpg
У839ВВ70    3 

https://storage.yandexcloud.net/plates/control_tasks/669_4818ае76_nan_nan.j

In [14]:
#агрегируем с помощью majority vote
res_groupby = res[['INPUT:image','OUTPUT:output']].groupby(['INPUT:image']).agg(lambda x:x.value_counts().index[0])

In [15]:
#соединим ответы крауда с ответами нейронки
df = pd.merge(df,res_groupby,how='left',left_on='url',right_index = True)

In [16]:
#в итоге наши толокеры исправили 50 номеров 
df[(df['is_good_prediction'] == 0) & (df['predict'] != df['OUTPUT:output']) & (~df['predict'].isnull())]

,id,url,predict,confidence,is_good_prediction,OUTPUT:output
68,39473.jpg,https://storage.yandexcloud.net/plates/39473.jpg,С643Н799,0.972318,0,С643МН799
102,49829.jpg,https://storage.yandexcloud.net/plates/49829.jpg,77ААS1,0.771649,0,6877АА51
146,23643.jpg,https://storage.yandexcloud.net/plates/23643.jpg,К271В,0.964197,0,3519АР50
150,17791.jpg,https://storage.yandexcloud.net/plates/17791.jpg,VУ68577,0.904899,0,УУ68577
175,15787.jpg,https://storage.yandexcloud.net/plates/15787.jpg,RТЕ66,0.846301,0,Е384ТУ33
230,64255.jpg,https://storage.yandexcloud.net/plates/64255.jpg,РК575Р7,0.925318,0,РК57577
244,35589.jpg,https://storage.yandexcloud.net/plates/35589.jpg,232УО102,0.572255,0,С232УО102
346,7411.jpg,https://storage.yandexcloud.net/plates/7411.jpg,V291077,0.761254,0,УХ291077
394,56929.jpg,https://storage.yandexcloud.net/plates/56929.jpg,С900МЕ46,0.847008,0,С900МЕ45
402,4873.jpg,https://storage.yandexcloud.net/plates/4873.jpg,С687ХЕ77,0.997915,0,С687ХЕ777


In [17]:
#запишем в answer наш итоговый ответ
df['answer'] = df['predict']
df.loc[df['is_good_prediction'] == 0,'answer'] = df['OUTPUT:output']

In [18]:
#сохраним нашу таблицу в файл answer.csv. Этот файл вы должны прислать мне 
df[['url','answer']].to_csv('data/answer.csv',index = None)

In [19]:
#этоот код вам не нужен, да и не запустится он у вас, так как ground_truth есть только у меня
#он необходим мне для проверки ваших решений
df = df[['url','answer','predict']]
df['id'] = df['url'].apply(lambda x: x.split('/')[-1])

In [24]:
#открываем ground_truth
ground_truth = pd.read_csv('../shad/test_solution/threshold05.csv')[['id','plate']]
ground_truth['plate'] = ground_truth['plate'].str.upper()

In [25]:
#мерджим с вашим вариантом
data = pd.merge(df,ground_truth,how='left',left_on='id',right_on = 'id')
data.head()

,url,answer,predict,id,plate
0,https://storage.yandexcloud.net/plates/30100.jpg,У004МУ30,У004МУ30,30100.jpg,У004МУ30
1,https://storage.yandexcloud.net/plates/54865.jpg,М998МР77,М998МР77,54865.jpg,М998МР77
2,https://storage.yandexcloud.net/plates/40585.jpg,К214МО57,NaN,40585.jpg,К214МО57
3,https://storage.yandexcloud.net/plates/34801.jpg,Р142ХО26,Р142ХО26,34801.jpg,Р142ХО26
4,https://storage.yandexcloud.net/plates/3061.jpg,А777КН67,А777КН67,3061.jpg,А777КН67


In [26]:
#считаем точность
accuracy = (data['answer'] == data['plate']).value_counts()
display(accuracy)
print(f'accuracy: {accuracy[True]/2000}')

True     1981
False      19
dtype: int64

accuracy: 0.9905


In [27]:
#считаем точность без использования крауда
accuracy_nn = (data['predict'] == data['plate']).value_counts()
display(accuracy_nn)
print(f'accuracy: {accuracy_nn[True]/2000}')

True     1711
False     289
dtype: int64

accuracy: 0.8555


In [29]:
data.head()

,url,answer,predict,id,plate
0,https://storage.yandexcloud.net/plates/30100.jpg,У004МУ30,У004МУ30,30100.jpg,У004МУ30
1,https://storage.yandexcloud.net/plates/54865.jpg,М998МР77,М998МР77,54865.jpg,М998МР77
2,https://storage.yandexcloud.net/plates/40585.jpg,К214МО57,NaN,40585.jpg,К214МО57
3,https://storage.yandexcloud.net/plates/34801.jpg,Р142ХО26,Р142ХО26,34801.jpg,Р142ХО26
4,https://storage.yandexcloud.net/plates/3061.jpg,А777КН67,А777КН67,3061.jpg,А777КН67


In [30]:
#примеры, в которых ответ не сошелся с ground_truth
for url,anser,_,_, plate in data[data['answer'] != data['plate']].itertuples(index=False):
    print(url, plate, anser)

https://storage.yandexcloud.net/plates/65.jpg О158МК198 О158НК198
https://storage.yandexcloud.net/plates/55836.jpg АО85799 А085799
https://storage.yandexcloud.net/plates/45462.jpg М871ОО05 Н871ОО05
https://storage.yandexcloud.net/plates/53921.jpg В037МС126 В037MС126
https://storage.yandexcloud.net/plates/26682.jpg О128НЕ750 К228РЕ750
https://storage.yandexcloud.net/plates/33086.jpg А555РЕ177 Н737РС97
https://storage.yandexcloud.net/plates/13162.jpg М494ОН68 М494СН68
https://storage.yandexcloud.net/plates/47249.jpg РО15477 Р015477
https://storage.yandexcloud.net/plates/38929.jpg М882РС32 М802РС32
https://storage.yandexcloud.net/plates/41986.jpg А892АС761 А892АС76
https://storage.yandexcloud.net/plates/60785.jpg А381СМ777 А381СМ77
https://storage.yandexcloud.net/plates/7839.jpg Н222ХУ37 Н629НН37
https://storage.yandexcloud.net/plates/63218.jpg АО849522 К404НХ76
https://storage.yandexcloud.net/plates/11402.jpg Е447ММ43 Е447МM43
https://storage.yandexcloud.net/plates/5282.jpg Е944ВК198 Е94